In [42]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embedding_function = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
from langchain.schema import Document
import json

<h2>LOADING THE SAVED CHUNKS AND THE CHROMA VECTOR DATABASE</h2>

In [43]:
def load_chunks(filename="preprocessed_chunks.json"):
    with open(filename, 'r') as f:
        chunk_dicts = json.load(f)
    return [Document(page_content=c["page_content"], metadata=c["metadata"]) for c in chunk_dicts]

# Load chunks instead of reprocessing PDF
chunks = load_chunks()

# Load the stored vector database
vectorstore = Chroma(persist_directory="./chroma_db_pedition", embedding_function=embedding_function)

<h2>USING BOTH VECTOR SIMILARITY AND KEYWORD SIMILARITY</h2>

In [44]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# vector retriever
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Keyword retriever (BM25)
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5

# Hybrid ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.7, 0.3]  # tune if needed
)

relevant_chunks = ensemble_retriever.get_relevant_documents("who wrote odyssey?")

In [19]:
# relevant_chunks

<h2>HYBRID APPROACH OF USING HYBRID RETRIEVAL APPROACH FOLLOWED BY RE-RANKING</h2>

In [45]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np


# Load cross-encoder model
# rerank_model_name = "BAAI/bge-reranker-large" # USE THIS IF NEEDED A PRECISE RESULT
rerank_model_name = "BAAI/bge-reranker-base"  # USE THIS IF NEEDED A FASTER RESULT
tokenizer = AutoTokenizer.from_pretrained(rerank_model_name)
model = AutoModelForSequenceClassification.from_pretrained(rerank_model_name)


from nltk.tokenize import sent_tokenize
from langchain.schema import Document
import nltk
nltk.download('punkt')

def rerank_chunks(query, chunks, top_k=3):
    # Split paragraphs into sentences while maintaining metadata
    sentences = []
    for chunk in chunks:
        try:
            chunk_sentences = sent_tokenize(chunk.page_content)
        except:
            # Fallback for simple sentence splitting if NLTK fails
            chunk_sentences = chunk.page_content.split('. ')
        
        for sent in chunk_sentences:
            sentences.append(Document(
                page_content=sent.strip(),
                metadata=chunk.metadata  # Preserve original metadata
            ))

    # Create query-sentence pairs for scoring
    pairs = [[query, doc.page_content] for doc in sentences]
    inputs = tokenizer(pairs, padding=True, truncation=True, 
                      return_tensors="pt", max_length=512)

    with torch.no_grad():
        scores = model(**inputs).logits.view(-1).float()

    # Sort sentences by their relevance scores
    sorted_indices = scores.argsort(descending=True)
    top_sentences = [sentences[i] for i in sorted_indices[:top_k]]
    top_scores = scores[sorted_indices[:top_k]].numpy()

    return top_sentences, top_scores


def calculate_confidence(scores):
    """Takes the scores from `rerank_chunks` and applies sigmoid to get a confidence score."""
    probabilities = torch.sigmoid(torch.tensor(scores)).numpy()
    return float(np.max(probabilities))  # Return the highest confidence score


# relevant_sentences, sentence_scores = rerank_chunks("who wrote odyssey?", relevant_chunks)
# final_confidence = calculate_confidence(sentence_scores)
# relevant_sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anandhu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
# initial_chunks = ensemble_retriever.get_relevant_documents("who wrote odyssey?")
# final_chunks, relevance_scores = rerank_chunks("who wrote odyssey?", initial_chunks, top_k=3)

# confidence = calculate_confidence(relevance_scores)
# final_chunks

# # 16.1 SECONDS FIRST RUN
# # 21.9 SECONDS SECOND RUN
# # 20.5 SECONDS THIRD RUN

In [22]:
# initial_chunks = ensemble_retriever.get_relevant_documents("Explain the themes of hospitality in The Odyssey.")
# final_chunks, relevance_scores = rerank_chunks("Explain the themes of hospitality in The Odyssey.", initial_chunks, top_k=3)
# final_chunks

# # 18.0 SECONDS FIRST RUN
# # 18.4 SECONDS SECOND RUN

<h2>LOADING LLAMA3 ( GROQ )</h2>

In [47]:
from langchain_groq import ChatGroq
from AI_GATEWAYS import groq_api_key

llm = ChatGroq(
    # model_name = "deepseek-r1-distill-llama-70b",
    model_name = "llama3-70b-8192",
    # model_name = "mixtral-8x7b-32768",
    temperature=0,
    groq_api_key = groq_api_key
)

<h2>LOADING LLAMA3 ( HUGGINGFACE )</h2>

In [46]:
# from langchain_community.llms import HuggingFaceHub
# from AI_GATEWAYS import huggingface_api_key

# llm = HuggingFaceHub(
#     repo_id="meta-llama/Llama-3.3-70B-Instruct",
#     model_kwargs={"temperature": 0.2, "max_length": 1024},
#     huggingfacehub_api_token=huggingface_api_key
# )

In [25]:
# from langchain_community.llms import HuggingFaceHub
# from AI_GATEWAYS import huggingface_api_key

# llm = HuggingFaceHub(
#     repo_id="deepseek-ai/DeepSeek-R1",
#     model_kwargs={"temperature": 0.2, "max_length": 1024},
#     huggingfacehub_api_token=huggingface_api_key
# )

C:\Users\Anandhu\AppData\Local\Temp\ipykernel_16388\3817004653.py:4: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [ ]:
# from langchain_community.llms import HuggingFaceHub
# from AI_GATEWAYS import huggingface_api_key

# llm = HuggingFaceHub(
#     repo_id="deepseek-ai/DeepSeek-R1",
#     model_kwargs={"temperature": 0.2, "max_length": 1024},
#     huggingfacehub_api_token=huggingface_api_key
# )

<h2>LLM IMPLEMENTATION USING MEMORY AND THRESHHOLD</h2>

In [48]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Create memory that retains last 3 exchanges
memory = ConversationBufferWindowMemory(
    k=10,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)


# Modified QA Chain with Memory
qa_prompt = ChatPromptTemplate.from_template(
    """Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    {chat_history}
    
    Context:
    {context}
    
    Question: {question}
    
    Answer in complete sentences, citing text evidence. 
    If unsure, say so:"""
)


qa_chain = (
    {"context": lambda x: x["chunks"], 
     "question": lambda x: x["question"],
     "chat_history": lambda x: x["chat_history"]}
    | qa_prompt
    | llm
    | StrOutputParser()
)

In [49]:
def format_chunks(chunks):
    return "\n\n".join([f"Page {c.metadata['page']}: {c.page_content}" for c in chunks])



def get_threshold_response():
    return "I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?"



# Modified ask_question function with confidence scoring
def ask_question(question, confidence_threshold=0.65):
    # Retrieve context
    initial_chunks = ensemble_retriever.get_relevant_documents(question)
    final_chunks, relevance_scores = rerank_chunks(question, initial_chunks, top_k=3)
    
    # Calculate confidence
    confidence = calculate_confidence(relevance_scores)
    
    # Generate answer
    answer = qa_chain.invoke({
        "question": question,
        "chunks": format_chunks(final_chunks),
        "chat_history": memory.load_memory_variables({})["chat_history"]
    })
    
    # Store interaction in memory
    memory.save_context({"question": question}, {"answer": answer})
    
    # Add confidence and sources
    sources = list(set(c.metadata["page"] for c in final_chunks))
    response = f"{answer}\n\nConfidence: {confidence:.0%}\nSources: Pages {', '.join(map(str, sources))}"
    
    return response if confidence >= confidence_threshold else f"{get_threshold_response()}\n\n{response}"

In [28]:
# # 1. IMPROVED PROMPT TEMPLATE (No chat history references)
# qa_prompt = ChatPromptTemplate.from_messages([
#     ("system", """You are an expert on Homer's Odyssey. Answer clearly and concisely using ONLY the context provided. If uncertain, state that explicitly.
    
#     Context:
#     {context}
    
#     Current Question: {question}"""),
#     ("human", "{question}")
# ])

# # 2. SIMPLIFIED MEMORY CONFIGURATION (Disable if not needed)
# memory = ConversationBufferWindowMemory(
#     k=3,  # Smaller window
#     memory_key="chat_history",
#     return_messages=False,  # Return string instead of messages
#     output_key="answer"
# )

# # 3. CONFIDENCE CALIBRATION
# def calculate_confidence(scores):
#     """Normalize scores between 0-1 using softmax"""
#     probabilities = torch.softmax(torch.tensor(scores), dim=0).numpy()
#     return float(np.max(probabilities))

# # 4. UPDATED ASK_QUESTION FUNCTION
# def ask_question(question, confidence_threshold=0.4):  # Lower threshold
#     # Retrieve and rerank chunks
#     initial_chunks = ensemble_retriever.get_relevant_documents(question)
#     final_chunks, relevance_scores = rerank_chunks(question, initial_chunks, top_k=3)
    
#     # Generate answer
#     answer = qa_chain.invoke({
#         "question": question,
#         "context": format_chunks(final_chunks),
#         "chat_history": memory.load_memory_variables({}).get("chat_history", "")
#     }).strip()  # Clean whitespace
    
#     # Calculate calibrated confidence
#     confidence = calculate_confidence(relevance_scores)
    
#     # Format response
#     sources = list(set(c.metadata["page"] for c in final_chunks))
#     base_response = f"{answer}\n\nConfidence: {confidence:.0%}\nSources: Pages {', '.join(map(str, sources))}"
    
#     return base_response if confidence >= confidence_threshold else f"Uncertain Response: {base_response}"

In [29]:
# print(ask_question("Who is Telemachus?"))

# # LLM DEEPSEEK

c:\Users\Anandhu\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

Human: Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    []
    
    Context:
    Page 282: Who has set my bed otherwhere?

Page 96: Who gave thee this raiment?

Page 195: Howbeit, Olympian Zeus, that dwells in the clear sky, knows hereof, whether or no he will fulfill for them the evil day before their marriage.” Now even as he spake, a bird flew out on the right, a hawk, the swift messenger of Apollo.
    
    Question: Who is Telemachus?
    
    Answer in complete sentences, citing text evidence. 
    If unsure, say so:

Confidence: 1%
Sources: Pages 96, 282, 195


In [30]:
# print(ask_question("And what about his relationship with Odysseus?"))

# LLM DEEPSEEK

c:\Users\Anandhu\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Human: Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    [HumanMessage(content='Who is Telemachus?', additional_kwargs={}, response_metadata={}), AIMessage(content='Human: Answer based on our conversation history and the Odyssey context:\n    \n    Chat History:\n    []\n    \n    Context:\n    Page 282: Who has set my bed otherwhere?\n\nPage 96: Who gave thee this raiment?\n\nPage 195: Howbeit, Olympian Zeus, that dwells in the clear sky, knows hereof, whether or no he will fulfill for them the evil day before their marriage.” Now even as he spake, a bird flew out on the right, a hawk, the swift messenger of Apollo.\n    \n    Question: Who is Telemachus?\n    \n    Answer in complete sentences, citing text evidence. \n    If unsure, say so:', additional_kwargs={}, response_metadata={})]
    
    Context:
    Page 292: Then he communed with his heart and soul, whether he should fall on his father’s neck and kiss him, and tell him all, how he 

In [31]:
# print(ask_question("Did Odysseus use a lightsaber?"))

# LLM DEEPSEEK

c:\Users\Anandhu\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

Human: Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    [HumanMessage(content='Who is Telemachus?', additional_kwargs={}, response_metadata={}), AIMessage(content='Human: Answer based on our conversation history and the Odyssey context:\n    \n    Chat History:\n    []\n    \n    Context:\n    Page 282: Who has set my bed otherwhere?\n\nPage 96: Who gave thee this raiment?\n\nPage 195: Howbeit, Olympian Zeus, that dwells in the clear sky, knows hereof, whether or no he will fulfill for them the evil day before their marriage.” Now even as he spake, a bird flew out on the right, a hawk, the swift messenger of Apollo.\n    \n    Question: Who is Telemachus?\n    \n    Answer in complete sentences, citing text evidence. \n    If unsure, say so:', additional_kwargs={}, response_metadata={}), HumanMessage(content='And what about his relationship w

In [39]:
# print(ask_question("Who is Telemachus?"))

# LLM LLAMA 70 B HUGGINGFACE

c:\Users\Anandhu\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

Human: Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    []
    
    Context:
    Page 282: Who has set my bed otherwhere?

Page 96: Who gave thee this raiment?

Page 195: Howbeit, Olympian Zeus, that dwells in the clear sky, knows hereof, whether or no he will fulfill for them the evil day before their marriage.” Now even as he spake, a bird flew out on the right, a hawk, the swift messenger of Apollo.
    
    Question: Who is Telemachus?
    
    Answer in complete sentences, citing text evidence. 
    If unsure, say so:

Confidence: 1%
Sources: Pages 96, 282, 195


In [40]:
# print(ask_question("And what about his relationship with Odysseus?"))

# LLM LLAMA 70 B HUGGINGFACE

c:\Users\Anandhu\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Human: Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    [HumanMessage(content='Who is Telemachus?', additional_kwargs={}, response_metadata={}), AIMessage(content='Human: Answer based on our conversation history and the Odyssey context:\n    \n    Chat History:\n    []\n    \n    Context:\n    Page 282: Who has set my bed otherwhere?\n\nPage 96: Who gave thee this raiment?\n\nPage 195: Howbeit, Olympian Zeus, that dwells in the clear sky, knows hereof, whether or no he will fulfill for them the evil day before their marriage.” Now even as he spake, a bird flew out on the right, a hawk, the swift messenger of Apollo.\n    \n    Question: Who is Telemachus?\n    \n    Answer in complete sentences, citing text evidence. \n    If unsure, say so:', additional_kwargs={}, response_metadata={})]
    
    Context:
    Page 292: Then he communed with his heart and soul, whether he should fall on his father’s neck and kiss him, and tell him all, how he 

In [41]:
# print(ask_question("Did Odysseus use a lightsaber?"))

# LLM LLAMA 70 B HUGGINGFACE

c:\Users\Anandhu\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

Human: Answer based on our conversation history and the Odyssey context:
    
    Chat History:
    [HumanMessage(content='Who is Telemachus?', additional_kwargs={}, response_metadata={}), AIMessage(content='Human: Answer based on our conversation history and the Odyssey context:\n    \n    Chat History:\n    []\n    \n    Context:\n    Page 282: Who has set my bed otherwhere?\n\nPage 96: Who gave thee this raiment?\n\nPage 195: Howbeit, Olympian Zeus, that dwells in the clear sky, knows hereof, whether or no he will fulfill for them the evil day before their marriage.” Now even as he spake, a bird flew out on the right, a hawk, the swift messenger of Apollo.\n    \n    Question: Who is Telemachus?\n    \n    Answer in complete sentences, citing text evidence. \n    If unsure, say so:', additional_kwargs={}, response_metadata={}), HumanMessage(content='And what about his relationship w

In [50]:
print(ask_question("Who is Telemachus?"))

# LLM LLAMA 70B GROQ

I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

Based on our conversation history and the Odyssey context, Telemachus is the son of Odysseus, as evidenced by the fact that he is concerned about his father's marriage and is present in the scenes where his father's belongings are being questioned, such as the bed and the raiment.

Confidence: 1%
Sources: Pages 96, 282, 195


In [51]:
print(ask_question("And what about his relationship with Odysseus?"))

# LLM LLAMA 70B GROQ

Telemachus is deeply concerned about his father Odysseus' well-being and is uncertain about how to approach him, as evidenced by the fact that he communed with his heart and soul about whether to fall on his father's neck and kiss him, or to first question him and prove him in every word (Page 292). This hesitation suggests a strong emotional bond between Telemachus and Odysseus, and Telemachus' desire to reconnect with his father.

Confidence: 98%
Sources: Pages 281, 292, 260


In [52]:
print(ask_question("Did Odysseus use a lightsaber?"))

# LLM LLAMA 70B GROQ

I'm not entirely confident about this answer. Would you like to rephrase or ask about another topic?

No, Odysseus did not use a lightsaber. There is no mention of lightsabers in the provided context, and the text only describes Odysseus using traditional ancient Greek armor and weapons, such as shields, spears, and helmets with horse hair crests (Page 268).

Confidence: 37%
Sources: Pages 268, 12
